In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier

In [8]:
def onehotmodel(data):
    num_col = data.select_dtypes(include=[np.number])
    non_num_col = data.select_dtypes(exclude=[np.number])
    onehotnum = pd.get_dummies(non_num_col)
    data = pd.concat([num_col, onehotnum], axis=1)
    return data


train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
test_CUST_ID = test_data['CUST_ID']
train_data_target = train_data['bad_good']
train_data.drop(['bad_good'], axis=1, inplace=True)
dropcols = [
    "OPEN_ORG_NUM", "IDF_TYP_CD", "GENDER", "CUST_EUP_ACCT_FLAG",
    "CUST_AU_ACCT_FLAG", "CUST_SALARY_FINANCIAL_FLAG",
    "CUST_SOCIAL_SECURITYIC_FLAG", "CUST_MTFLOW_FLAG", "CUST_DOLLER_FLAG",
    "CUST_INTERNATIONAL_GOLD_FLAG", "CUST_INTERNATIONAL_GOLD_FLAG",
    "CUST_INTERNATIONAL_SIL_FLAG", "CUST_INTERNATIONAL_DIAMOND_FLAG",
    "CUST_GOLD_COMMON_FLAG", "CUST_STAD_PLATINUM_FLAG",
    "CUST_LUXURY_PLATINUM_FLAG", "CUST_PLATINUM_FINANCIAL_FLAG",
    "CUST_DIAMOND_FLAG", "CUST_INFINIT_FLAG", "CUST_BUSINESS_FLAG"
]
train_data.drop(dropcols, axis=1, inplace=True)
test_data.drop(dropcols, axis=1, inplace=True)
train_data = train_data.drop_duplicates(keep="first")
train_data.dropna(inplace=True)
train_data = onehotmodel(train_data)
test_data = onehotmodel(test_data)
x_train, x_test, y_train, y_test = train_test_split(train_data,
                                                    train_data_target,
                                                    test_size=0.2,
                                                    random_state=42)

C:\Users\jy\AppData\Local\Temp\ipykernel_20352\2463252460.py:9: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('train.csv')
C:\Users\jy\AppData\Local\Temp\ipykernel_20352\2463252460.py:10: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv('test.csv')


In [12]:
XGB = XGBClassifier(learning_rate=0.08,
                    n_estimators=50,
                    max_depth=5,
                    gamma=0,
                    subsample=0.9,
                    colsample_bytree=0.5)
model = XGB.fit(x_train, y_train)
y_pred = model.predict(x_test)
print("验证数据的准确率: ", accuracy_score(y_test, y_pred))
print("验证数据的精确率 ", precision_score(y_test, y_pred))
print("验证数据的召回率 ", recall_score(y_test, y_pred))
print("验证数据的F1值 ", f1_score(y_test, y_pred))
test_pred = model.predict(test_data)
test_pred = pd.DataFrame(test_pred, columns=['bad_good'])
sub = pd.concat([test_CUST_ID, test_pred], axis=1)
sub.to_csv('submission.csv')

验证数据的准确率:  0.9999649473333684
验证数据的精确率  0.9978540772532188
验证数据的召回率  1.0
验证数据的F1值  0.9989258861439313
